# Final ML Project
## Daniel Bernal, Raymond Vuong, Rohit Punjani, and Neal Davar 


In [249]:
# import statements

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [250]:
# Constants and other global variables: 

IL_F_ITERATIONS = 50

In [251]:
# clean data by dropping cols like Id, imputing value
# depending on the feature, and dropping duplicates:
def clean_data(dataset, is_training): 
    dataset.drop('Id', axis=1, inplace=True)

    if is_training: 
        dataset = dataset.drop('SalePrice', axis=1, inplace=False)
    
    null_counts = dataset.isnull().sum()
    missing_features = null_counts[null_counts > 0]
    print(missing_features)    
    
   
    # fillna on the basis of whether MasVnrArea, LotFrontage, or any other categorical feature with na values
    dataset['MSZoning'].fillna('None', inplace=True)
    dataset['LotFrontage'].fillna(0, inplace=True)
    dataset['Alley'].fillna('None', inplace=True)
    dataset['Utilities'].fillna('None', inplace=True)
    dataset['Exterior1st'].fillna('None', inplace=True)
    dataset['Exterior2nd'].fillna('None', inplace=True)
    dataset['MasVnrType'].fillna('None', inplace=True)
    dataset['MasVnrArea'].fillna(0, inplace=True)
    dataset['BsmtQual'].fillna('None', inplace=True)
    dataset['BsmtCond'].fillna('None', inplace=True)
    dataset['BsmtExposure'].fillna('None', inplace=True)
    dataset['BsmtFinType1'].fillna('None', inplace=True)
    dataset['BsmtFinSF1'].fillna(0, inplace=True)
    dataset['BsmtFinType2'].fillna('None', inplace=True)
    dataset['BsmtFinSF2'].fillna(0, inplace=True)
    dataset['BsmtUnfSF'].fillna(0, inplace=True)
    dataset['TotalBsmtSF'].fillna(0, inplace=True)
    dataset['BsmtFullBath'].fillna(0, inplace=True)
    dataset['BsmtHalfBath'].fillna(0, inplace=True)
    dataset['KitchenQual'].fillna('None', inplace=True)
    dataset['Functional'].fillna('None', inplace=True)
    dataset['FireplaceQu'].fillna('None', inplace=True)
    dataset['GarageType'].fillna('None', inplace=True)
    dataset['GarageYrBlt'].fillna(dataset['GarageYrBlt'].mean(), inplace=True)
    dataset['GarageFinish'].fillna('None', inplace=True)
    dataset['GarageCars'].fillna(0, inplace=True)
    dataset['GarageArea'].fillna(0, inplace=True)
    dataset['GarageQual'].fillna('None', inplace=True)
    dataset['GarageCond'].fillna('None', inplace=True)
    dataset['PoolQC'].fillna('None', inplace=True)
    dataset['Fence'].fillna('None', inplace=True)
    dataset['MiscFeature'].fillna('None', inplace=True)
    dataset['SaleType'].fillna('None', inplace=True)

    # drop duplicates:
    dataset.drop_duplicates(keep=False, inplace=True)

    # Changing categorial features to be stored as string
    dataset['MSSubClass'] = dataset['MSSubClass'].astype(str)

    print(((is_training and "Training: ") or (not is_training and "Testing: ")), dataset.describe)

    return dataset

In [252]:
# on hot encode the data
def one_hot_encode(dataset):
    ohe_col_list = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
                    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']   

    dataset_dropped = pd.get_dummies(data=dataset, columns=ohe_col_list, drop_first=True)

    return dataset_dropped

In [253]:
def explore_data(dataset):
    # grab all the numeric features and plot histograms
    numeric_feats = dataset.select_dtypes(
    include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
    for i in numeric_feats.columns:
        plt.hist(numeric_feats[i])
        plt.title(i)
        plt.show()

    # plot correlation between features: 
    corr_mat = dataset.corr()
    plt.subplots(figsize=(12, 9))
    sns.heatmap(corr_mat, square=True)

In [254]:
# TODO: ASK ANSHUL!!
def one_hot_encoder(training, testing):
   
    ohe = OneHotEncoder(handle_unknown='ignore', drop='first')
    labels = training['SalePrice']
    training_features = training.drop('SalePrice', axis=1)
    ohe.fit(training_features)
    transform_training = pd.DataFrame(ohe.transform(training_features).toarray())
    print(ohe.get_feature_names_out())
    transform_testing = pd.DataFrame(ohe.transform(testing).toarray())

    pd.concat([transform_training, labels], axis=1)

    print(transform_training)
    print(transform_testing)

    return transform_training, transform_testing


In [270]:
# Uses the isolation forest technique to find outliers across 50 iterations
# and removes records that are recorded as outliers > 10% of the time
def run_isolation_forest(training):
    # use isolation forests to find potential outliers:
    freq_outlier_map = {} 
    for i in range(0, IL_F_ITERATIONS):
        anomalies = IsolationForest().fit_predict(training, 0.5)
        training['anomalies'] = anomalies
        outlier_indices = training.loc[training['anomalies'] == -1].index
        # add outlier freqs to map
        for j in range(0, len(outlier_indices)):
            count = 0
            if outlier_indices[j] in freq_outlier_map: 
                count = freq_outlier_map[outlier_indices[j]]
            freq_outlier_map[outlier_indices[j]] = count + 1
        

        
    # drop outliers that are detected as anomalies more than 10% of the time
    final_outlier_indices = []
    print('Total # of Outliers: ')
    for outlier_index in freq_outlier_map.keys(): 
        if freq_outlier_map[outlier_index] > (0.10 * IL_F_ITERATIONS):
            final_outlier_indices.append(outlier_index)
    print(len(final_outlier_indices))
    print('Number of data points before outlier removal: ')
    print(len(training))
    training.drop(index=final_outlier_indices, inplace=True)
    print('Number of data points before after outlier removal: ')
    print(len(training))
    training.drop('anomalies', axis=1, inplace=True)


In [256]:
# function that normalizes the training and testing data:
def normalize_data(train, test):
    # your code goes here
    train_norm = (train - train.min()) / (train.max() - train.min())
    test_norm = (test - test.min()) / (test.max() - test.min())
    return train_norm, test_norm

In [257]:
# feature engineering of consolidating bathroom
def fe_bathrooms(dataset):
    # consolidate bathroom columns into one col:
    dataset['TotalBathrooms'] = \
        dataset['FullBath'] + (0.5 * dataset['HalfBath']) + \
        dataset['BsmtFullBath'] + (0.5 * dataset['BsmtHalfBath'])
    dataset.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis=1)

In [258]:
# feature engineering of consolidating porch sq ft: 
def fe_porch(dataset):
    # consolidate porch area into one column
    dataset['TotalPorchSF'] = dataset['OpenPorchSF'] + \
        dataset['EnclosedPorch'] + \
        dataset['3SsnPorch'] + dataset['ScreenPorch']

    dataset.drop(['OpenPorchSF', 'EnclosedPorch',
                  '3SsnPorch', '3SsnPorch'], axis=1)


In [259]:
# feature engineering to apply log function
def fe_log(dataset, cols):
    for col in cols: 
        dataset[col] = np.log(dataset[col])

In [260]:
# Run Decision Tree Regression on our Data:
def do_decision_tree_regression(training):
    labels = training['SalePrice']
    features = training.drop('SalePrice', axis=1)
    pca = PCA()
    dtr = DecisionTreeRegressor()
    pipeline = Pipeline(
        steps=[('pca', pca), ('dt', dtr)])
    param_grid = {'pca__n_components': list(range(5, 20)),
                  'dt__max_depth': [5, 10, 15, 20, 30, 40], 'dt__min_samples_leaf': [
        5, 10, 15, 20, 30, 40], 'dt__max_features': [5, 10, 15, 30, 40]}

    grid = GridSearchCV(pipeline, param_grid, scoring='r2', cv=5)
    grid.fit(features, labels)
    print("DECISION TREE REGRESSION MODEL: ")
    print("Best number of dimensions: ",
          grid.best_params_['pca__n_components'])
    print("Best parameters: ", grid.best_params_)
    # nested cross val
    cv = cross_val_score(grid, features, labels, cv=5)
    r2 = sum(cv)/cv.size
    print("R2 with cross val: ", r2)


In [261]:
# run adaboost regression on training data
def do_adaBoost(training):
    labels = training['SalePrice']
    features = training.drop('SalePrice', axis=1)
    pca = PCA()
    ada = AdaBoostRegressor()
    pipeline = Pipeline(steps=[('pca', pca), ('ada', ada)])
    param_grid = {'pca__n_components': list(range(5, 12)), 'ada__n_estimators': [10, 20, 30,
                                    40, 50, 60]}

    grid = GridSearchCV(pipeline, param_grid, scoring='r2', cv=5)
    grid.fit(features, labels)
    print("ADA BOOST MODEL: ")
    print("Best number of dimensions: ",
          grid.best_params_['pca__n_components'])
    print("Best parameters: ", grid.best_params_)
    # nested cross val
    cv = cross_val_score(grid, features, labels, cv=5)
    r2 = sum(cv)/cv.size
    print("R2 with cross val: ", r2)


In [271]:
# KNN Regression Model Implementation:
#KNN regression
def do_KNN_regression(training, testing):
    labels = training['SalePrice']
    features = training.drop('SalePrice', axis=1, inplace=False)
    testing_labels = testing['SalePrice']
    testing_features = testing.drop('SalePrice', axis=1, inplace=False)
    print(testing.shape)
    print(training.shape)
    pca = PCA()
    knn = KNeighborsRegressor()
    pipeline = Pipeline(steps=[('pca', pca), ('knn', knn)])
    param_grid = {'pca__n_components': list(range(5, 12)), 'knn__n_neighbors': [
        5, 10, 15, 20, 30, 40]}

    grid = GridSearchCV(pipeline, param_grid, scoring='r2', cv=5)
    grid.fit(features, labels)
    print("KNN REGRESSION MODEL: ")
    print("Best number of dimensions: ",
          grid.best_params_['pca__n_components'])
    print("Best parameters: ", grid.best_params_)
    # nested cross val
    cv = cross_val_score(grid, features, labels, cv=5)
    r2 = sum(cv)/cv.size
    print("R2 with cross val: ", r2)
    test_accuracy = grid.score(testing_features, testing_labels)
    print("Accuracy after running model on test set: ", test_accuracy)



In [263]:
# run random forest algorithm on dataset: 
def do_random_forest_regression(training):
  labels = training['SalePrice']
  features = training.drop('SalePrice', axis=1)
  dt = RandomForestRegressor()
  pca = PCA()
  pipeline = Pipeline(steps=[('pca', pca), ('dt', dt)])
  #innerloop of crossval
  param_grid = {'pca__n_components': list(range(5, 12)), 
                'dt__max_depth': [5, 10, 15, 20, 30, 40], 'dt__min_samples_leaf': [
      5, 10, 15, 20, 30, 40], 'dt__max_features': [5, 10, 15, 30, 40]}
  grid = GridSearchCV(pipeline,
                      param_grid, cv=5, scoring='r2')
  grid.fit(features, labels)
  print("RANDOM FOREST REGRESSION MODEL: ")
  print("Best parameters: ", grid.best_params_)
  cv = cross_val_score(grid, features, labels, cv=5)
  r2 = sum(cv)/cv.size
  # TODO: Uncomment this: 
  #   Scores[0,0] = r2
  print("R2 with cross val: ", r2)

In [264]:
# run gradient boosting regression on dataset
def do_gradient_boosting_regression(training, testing):
  labels = training['SalePrice']
  features = training.drop('SalePrice', axis = 1)
  testing_labels = testing['SalePrice']
  testing_features = testing.drop('SalePrice', axis = 1)
  gbr = GradientBoostingRegressor()
  pca = PCA()
  pipeline = Pipeline(steps=[('pca', pca), ('gbr', gbr)])

  #innerloop of crossval
  param_grid = {'pca__n_components': list(range(10, 15)), 'gbr__max_depth': [2, 4, 6, 8], 'gbr__min_samples_leaf': [2, 3, 5, 7], 'gbr__max_features': [3, 5, 7, 9]}
  grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
  grid.fit(features, labels)
  print("GRADIENT BOOSTING MODEL: ")
  print("Best parameters: ", grid.best_params_)
  cv = cross_val_score(grid, features, labels, cv = 5)
  r2 = sum(cv)/cv.size
  y_pred = grid.best_estimator_.predict(testing_features)

  # # plot the results
  # plt.scatter(testing_features, testing_labels, color='black')
  # plt.plot(testing_features, y_pred, color='blue', linewidth=3)
  # plt.xticks(())
  # plt.yticks(())
  # plt.show()


# TODO: Uncomment this:
#   Scores[0,2] = r2
  print("R2 with cross val", r2)

In [265]:
# run a linear regression model
def runLinReg(train_norm):
    labels = train_norm['SalePrice']
    labels.values.ravel()
    features = train_norm.drop('SalePrice', axis=1)
    print(features.shape)
    print(labels.shape)
    print(features.head())
    print(labels.head())
    pca = PCA()
    linReg = LinearRegression() 
    pipeline = Pipeline(steps=[('pca', pca), ('linreg', linReg)])
    scores = cross_val_score(pipeline, features, labels, cv=10)
    print("Accuracy:", scores.mean()*100)


In [266]:
# run all models on data
def run_models(dataset):
    print("LINEAR REGRESSION: ")
    runLinReg(dataset)
    print("KNN REGRESSION: ")
    do_KNN_regression(dataset)
    print("ADABOOST REGRESSION: ")
    do_adaBoost(dataset)
    print("DECISION TREE REGRESSION: ")
    do_decision_tree_regression(dataset)

In [272]:
# the kaggle dataset testing data does not have labels
# so we split the train into train and test:
all_data = pd.read_csv("train.csv")
training, testing = train_test_split(all_data, test_size=0.50)
training_labels = training['SalePrice']
testing_labels = testing['SalePrice']

# 1. Clean Data
training = clean_data(training, True)
testing = clean_data(testing, False)



#2. Explore Data
# TODO: uncomment this line
# explore_data(training)

#  one hot encode and do an inner join to make sure we have the same shape in test and train
ohe_train = one_hot_encode(training)
ohe_test = one_hot_encode(testing)
final_train, final_test = ohe_train.align(ohe_test, join='inner', axis=1)


# 3. Normalization
train_norm, test_norm = normalize_data(final_train, final_test)

# outlier detection:
run_isolation_forest(train_norm)

# 4. Feature Engineering: 

train_norm['SalePrice'] = training_labels
test_norm['SalePrice'] = testing_labels

# 4. Run Models
# run_models(train_norm)
# do_gradient_boosting_regression(train_norm, test_norm)
do_KNN_regression(train_norm, test_norm)

LotFrontage     128
Alley           677
MasVnrType        5
MasVnrArea        5
BsmtQual         16
BsmtCond         16
BsmtExposure     16
BsmtFinType1     16
BsmtFinType2     17
FireplaceQu     318
GarageType       36
GarageYrBlt      36
GarageFinish     36
GarageQual       36
GarageCond       36
PoolQC          727
Fence           587
MiscFeature     701
dtype: int64
Training:  <bound method NDFrame.describe of      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
892          20       RL         70.0     8414   Pave  None      Reg   
453          60       FV         75.0     9000   Pave  None      Reg   
465         120       RM          0.0     3072   Pave  None      Reg   
100          20       RL          0.0    10603   Pave  None      IR1   
289          70       RL         60.0     8730   Pave  None      Reg   
85           60       RL        121.0    16059   Pave  None      Reg   
1137         50       RL         54.0     6342   Pave  None      Reg   
198   